In [1]:
import numpy as np
import pandas as pd
import datetime
import pytz

# Load dataset

In [2]:
df = pd.read_csv('delivery_orders_march.csv')

In [3]:
sla = pd.read_excel('SLA_matrix.xlsx')

# Analytics

Maximum of 2 attempts. Sunday is rest day. 2nd attempt must be within 3 days of first attempt. SLA calculation begins from the next day after pickup (Day 0 = Day of Pickup; Day 1 = Next Day after Pickup)

Public Holidays:
- 2020-03-08 (Sunday);
- 2020-03-25 (Wednesday);
- 2020-03-30 (Monday);
- 2020-03-31 (Tuesday)

In [4]:
sla=sla.iloc[:5,1:6]
sla.columns=sla.iloc[0]
sla.drop(sla.index[0],inplace=True)
sla=pd.melt(sla, id_vars=np.nan, value_vars=['Metro Manila','Luzon','Visayas', 'Mindanao'], value_name='days')
sla.rename(columns={np.nan: 'origin', 0:'destination'}, inplace=True)
sla.days=sla['days'].apply(lambda x: int(x.split()[0]))

In [5]:
sla

,origin,destination,days
0,Metro Manila,Metro Manila,3
1,Luzon,Metro Manila,5
2,Visayas,Metro Manila,7
3,Mindanao,Metro Manila,7
4,Metro Manila,Luzon,5
5,Luzon,Luzon,5
6,Visayas,Luzon,7
7,Mindanao,Luzon,7
8,Metro Manila,Visayas,7
9,Luzon,Visayas,7


In [6]:
#region
df['origin'] = df.selleraddress.apply(lambda x: x.split()[-1].lower())
df['dest'] = df.buyeraddress.apply(lambda x: x.split()[-1].lower())

# get SLA
df['SLA'] = 7
df.loc[np.where((df.dest=='luzon') & (df.origin.isin(['luzon', 'manila'])), True, False), 'SLA'] = 5
df.loc[np.where((df.origin=='manila') & (df.dest.isin(['manila'])),True, False), 'SLA'] = 3

# drop buyer address and selleraddress
df.drop(['buyeraddress','selleraddress'], axis = 1, inplace = True)

In [7]:
# Convert epoch time to dates (with time removed)
for col in ['pick','1st_deliver_attempt','2nd_deliver_attempt']:
    df.loc[df[col].notna(),col] = df.loc[df[col].notna(),col].apply(lambda x: datetime.datetime.fromtimestamp(x, tz= pytz.timezone('Asia/Singapore')).date())

HOLIDAYS = pd.Series([datetime.date(2020, 3, 8), datetime.date(2020, 3, 25), datetime.date(2020, 3, 30), datetime.date(2020, 3, 31)]).values.astype('datetime64[D]')

def calc_bdays(samp, col1, col2):
    return np.busday_count(samp[col1].values.astype('datetime64[D]'),samp[col2].values.astype('datetime64[D]'),weekmask='1111110',holidays=HOLIDAYS)

# Business days
df['1st_bdays'] = calc_bdays(df, 'pick', '1st_deliver_attempt')
df.loc[df['2nd_deliver_attempt'].notna(),'2nd_bdays'] = calc_bdays(df.loc[df['2nd_deliver_attempt'].notna()], '1st_deliver_attempt', '2nd_deliver_attempt')

In [8]:
# Late for first and second
df.loc[df.loc[:,'1st_bdays'] > df.SLA, 'first_late'] = 1
df.loc[df.loc[:,'2nd_bdays'] > 3, 'second_late'] = 1
df = df.fillna(0)
df.loc[((df.first_late == 0) & (df.second_late == 0)), 'is_late'] = 0
df = df.fillna(1)

In [9]:
# Final df
df.is_late = df.is_late.apply(lambda x: int(x))
final_df = (df.loc[:,['orderid', 'is_late']].set_index('orderid')) 

In [10]:
final_df.is_late.value_counts()

0    2413891
1     762422
Name: is_late, dtype: int64

In [11]:
# Export
final_df.to_csv('31GB_pat.csv')

# Finding edge cases

In [69]:
# Check where pick up days are sundays (weekday = 6)
df['pickup_week'] = df.pick.apply(lambda x: x.weekday())

In [70]:
df[df.pickup_week == 6]

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,origin,dest,SLA,1st_bdays,2nd_bdays,first_late,second_late,is_late,pickup_week
3886,2258569257,2020-03-08,2020-03-11,0,manila,manila,3,2,0.0,0.0,0.0,0,6
4021,2266583863,2020-03-08,2020-03-10,2020-03-11,manila,manila,3,1,1.0,0.0,0.0,0,6
4044,2266698376,2020-03-08,2020-03-11,2020-03-18,manila,luzon,5,2,6.0,0.0,1.0,1,6
4328,2269070682,2020-03-08,2020-03-10,0,manila,manila,3,1,0.0,0.0,0.0,0,6
4415,2269814379,2020-03-08,2020-03-10,0,manila,luzon,5,1,0.0,0.0,0.0,0,6
7378,2296378148,2020-03-15,2020-03-21,0,manila,manila,3,5,0.0,1.0,0.0,1,6
7396,2296494021,2020-03-15,2020-03-17,0,manila,luzon,5,1,0.0,0.0,0.0,0,6
7560,2297805745,2020-03-15,2020-03-19,2020-03-23,manila,luzon,5,3,3.0,0.0,0.0,0,6
7614,2298239745,2020-03-15,2020-03-19,2020-03-21,manila,manila,3,3,2.0,0.0,0.0,0,6
7838,2298958949,2020-03-15,2020-03-18,2020-03-19,manila,manila,3,2,1.0,0.0,0.0,0,6


In [83]:
# Check buyer and seller
df_original['buyer_metro'] = df_original.buyeraddress.apply(lambda x: 'metro manila' in x.lower()) *1
df_original['buyer_l'] = df_original.buyeraddress.apply(lambda x: 'luzon' in x.lower()) *1
df_original['buyer_v'] = df_original.buyeraddress.apply(lambda x: 'visayas' in x.lower())*1
df_original['buyer_m'] = df_original.buyeraddress.apply(lambda x: 'mindanao' in x.lower())*1

In [110]:
# Check 2nd delivery attempt
df[df.loc[:,'2nd_bdays'] == 3]

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,origin,dest,SLA,1st_bdays,2nd_bdays,first_late,second_late,is_late,pickup_week
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3,2,3.0,0.0,0.0,0,2
6,2222738456,2020-03-02,2020-03-05,2020-03-09,manila,manila,3,3,3.0,0.0,0.0,0,0
8,2224704587,2020-03-04,2020-03-05,2020-03-09,manila,luzon,5,1,3.0,0.0,0.0,0,2
24,2227831996,2020-03-02,2020-03-06,2020-03-10,manila,manila,3,4,3.0,1.0,0.0,1,0
108,2228351279,2020-03-02,2020-03-07,2020-03-11,manila,luzon,5,5,3.0,0.0,0.0,0,0
127,2228414527,2020-03-04,2020-03-07,2020-03-11,manila,manila,3,3,3.0,0.0,0.0,0,2
162,2228582233,2020-03-04,2020-03-06,2020-03-10,manila,luzon,5,2,3.0,0.0,0.0,0,2
168,2228597328,2020-03-02,2020-03-07,2020-03-11,manila,visayas,7,5,3.0,0.0,0.0,0,0
179,2228655368,2020-03-05,2020-03-19,2020-03-22,manila,visayas,7,12,3.0,1.0,0.0,1,3
225,2228868351,2020-03-02,2020-03-04,2020-03-07,manila,luzon,5,2,3.0,0.0,0.0,0,0


In [120]:
df_original['pick'].apply(lambda x: datetime.datetime.fromtimestamp(x, tz= pytz.timezone('Asia/Singapore')).date())

0          2020-03-02
1          2020-03-04
2          2020-03-04
3          2020-03-05
4          2020-03-04
5          2020-03-04
6          2020-03-02
7          2020-03-02
8          2020-03-04
9          2020-03-04
10         2020-03-07
11         2020-03-02
12         2020-03-04
13         2020-03-04
14         2020-03-02
15         2020-03-02
16         2020-03-02
17         2020-03-02
18         2020-03-02
19         2020-03-02
20         2020-03-04
21         2020-03-02
22         2020-03-02
23         2020-03-02
24         2020-03-02
25         2020-03-06
26         2020-03-04
27         2020-03-02
28         2020-03-04
29         2020-03-02
              ...    
3176283    2020-04-04
3176284    2020-04-02
3176285    2020-04-03
3176286    2020-04-03
3176287    2020-04-03
3176288    2020-04-03
3176289    2020-04-03
3176290    2020-04-02
3176291    2020-04-02
3176292    2020-04-02
3176293    2020-04-03
3176294    2020-04-03
3176295    2020-04-04
3176296    2020-04-04
3176297   